<a href="https://colab.research.google.com/github/veritaem/Project2/blob/master/Kaggle_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#first up is some imports

In [1]:
!pip install category_encoders

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer, PolynomialFeatures
from sklearn.metrics import precision_score, r2_score, mean_squared_error, classification_report, confusion_matrix, mean_absolute_error, recall_score, accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, f_regression, f_classif
import seaborn as sns
import category_encoders as ce
import xgboost as xgb

In [0]:
pd.set_option('display.height', 3000)
pd.set_option('display.max_rows', 3000)
pd.set_option('display.max_columns', 3000)
pd.set_option('display.width', 3000)

In [0]:
df1 = pd.read_csv('https://raw.githubusercontent.com/veritaem/Project2/master/train_features.csv')
df2 = pd.read_csv('https://raw.githubusercontent.com/veritaem/Project2/master/train_labels.csv')
testdf = pd.read_csv('https://raw.githubusercontent.com/veritaem/Project2/master/test_features.csv')

#next up is to look at the data

In [5]:
df1.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [6]:
df2.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [7]:
df = df1.join(df2, rsuffix = 'status_group')
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,idstatus_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,69572,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,8776,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,34310,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,67743,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,19728,functional


In [0]:
id = pd.DataFrame(testdf['id'])

In [9]:
df.shape

(59400, 42)

In [10]:
df['status_group'].unique()

array(['functional', 'non functional', 'functional needs repair'],
      dtype=object)

In [11]:
df.isna().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [12]:
testdf.shape

(14358, 40)

In [0]:
nan_values_list = ['Not Known', 'Unknown', 'None', 'Not known', 'not known',
                  '-', 'unknown', 'Unknown Installer', '##', 'none', '0']

df = df.replace(nan_values_list, np.nan)
testdf = testdf.replace(nan_values_list, np.nan)

df = df.replace(np.nan, 'unknown')
testdf = testdf.replace(np.nan, 'unknown')

In [14]:
df.isna().sum()

id                       0
amount_tsh               0
date_recorded            0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
num_private              0
basin                    0
subvillage               0
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
public_meeting           0
recorded_by              0
scheme_management        0
scheme_name              0
permit                   0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
s

In [15]:
yencode = df['status_group'].replace({'functional':2, 'functional needs repair':1, 'non functional':0})
yencode = pd.DataFrame(yencode)
yencode.head()

,status_group
0,2
1,2
2,2
3,0
4,2


In [0]:
'''because certain columns seem unimportant, redundant, 
or missing most of their values, 
I will drop them in this cell, to be modified as needed later'''
df2 = df.drop(['scheme_name', 'public_meeting', 'payment_type', 'region', 'idstatus_group', 'recorded_by', 'wpt_name', 'latitude', 'longitude'],axis = 1)
testdf = testdf.drop(['scheme_name', 'public_meeting', 'payment_type', 'region', 'recorded_by', 'wpt_name', 'latitude', 'longitude'],axis = 1)

# ok, time for baselines


In [17]:
yencode.status_group.mean()

1.1588383838383838

In [18]:
baseline = [yencode.status_group.mean()] * len(yencode.status_group)  
mean_absolute_error(yencode.status_group, baseline)

0.9136374604632181

In [19]:
maj_classification = yencode.status_group.mode()
y_pred = np.full(shape=yencode['status_group'].shape, fill_value=maj_classification)
recall_score(yencode['status_group'], y_pred, average = 'micro')

0.543080808080808

#bins and engineering?

In [20]:
'''I see two values that would really ebnefit from a bin, and those are 
population and construction year, lets do it!'''

'I see two values that would really ebnefit from a bin, and those are \npopulation and construction year, lets do it!'

In [0]:
year_bins = [-1, 1980, 1990, 2000, 2010, 2020]
year_labels = [1, 2, 3, 4, 5]
df2['year_made'] = pd.cut(df2['construction_year'], bins = year_bins, labels = year_labels)
testdf['year_made'] = pd.cut(testdf['construction_year'], bins = year_bins, labels = year_labels)

In [0]:
pop_bins = [-1, 10, 20, 100, 250, 1000, 5000, 10000, 100000]
pop_labels = [1, 2, 3, 4, 5, 6, 7, 8]
df2['pop'] = pd.cut(df2['population'], bins = pop_bins, labels = pop_labels)
testdf['pop'] = pd.cut(testdf['population'], bins = pop_bins, labels = pop_labels)

Alright, lets see what we can see! *woosh woosh woosh*

In [23]:
df2.describe(include = 'all')

,id,amount_tsh,date_recorded,funder,gps_height,installer,num_private,basin,subvillage,region_code,district_code,lga,ward,population,scheme_management,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group,year_made,pop
count,59400.000000,59400.000000,59400,59400,59400.000000,59400,59400.000000,59400,59400,59400.000000,59400.000000,59400,59400,59400.000000,59400,59400,59400.000000,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400.0,59400.0
unique,NaN,NaN,356,1894,NaN,2139,NaN,9,19287,NaN,NaN,125,2092,NaN,12,3,NaN,18,13,7,12,5,7,8,6,5,5,10,7,3,7,6,3,5.0,8.0
top,NaN,NaN,2011-03-15,Government Of Tanzania,NaN,DWE,NaN,Lake Victoria,Madukani,NaN,NaN,Njombe,Igosi,NaN,VWC,True,NaN,gravity,gravity,gravity,vwc,user-group,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional,1.0,1.0
freq,NaN,NaN,572,9084,NaN,17402,NaN,10248,508,NaN,NaN,2503,307,NaN,36793,38852,NaN,26780,26780,26780,40507,52490,25348,50818,50818,33186,33186,17021,17021,45794,28522,34625,32259,26464.0,28690.0
mean,37115.131768,317.650385,NaN,NaN,668.297239,NaN,0.474141,NaN,NaN,15.297003,5.629747,NaN,NaN,179.909983,NaN,NaN,1300.652475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,21453.128371,2997.574558,NaN,NaN,693.116350,NaN,12.236230,NaN,NaN,17.587406,9.633649,NaN,NaN,471.482176,NaN,NaN,951.620547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,0.000000,NaN,NaN,-90.000000,NaN,0.000000,NaN,NaN,1.000000,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,18519.750000,0.000000,NaN,NaN,0.000000,NaN,0.000000,NaN,NaN,5.000000,2.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,37061.500000,0.000000,NaN,NaN,369.000000,NaN,0.000000,NaN,NaN,12.000000,3.000000,NaN,NaN,25.000000,NaN,NaN,1986.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,55656.500000,20.000000,NaN,NaN,1319.250000,NaN,0.000000,NaN,NaN,17.000000,5.000000,NaN,NaN,215.000000,NaN,NaN,2004.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df2['installer'].value_counts()
# i wanted to reduce the number of uniwue variables in each column, and it looks like a jackpot!
#a bunch of these look similar to each other (i.e. gover, government of Tanzania, Gover Dept water, etc)


DWE                               17402
unknown                            4481
Government                         1825
RWE                                1206
Commu                              1060
DANIDA                             1050
KKKT                                898
Hesawa                              840
TCRS                                707
Central government                  622
CES                                 610
Community                           553
DANID                               552
District Council                    551
HESAWA                              539
LGA                                 408
World vision                        408
WEDECO                              397
TASAF                               396
District council                    392
Gover                               383
AMREF                               329
TWESA                               316
WU                                  301
Dmdd                                287


In [0]:
red_list = ['Government', 'Central government', 'Central Govt', 'GOVER', 'Gover',
            'Gove', 'Centr', 'Central Government', 'Wizara ya maji', 'Water', 
            'GOVERNMENT', 'Tanzania Government', 'GOVERN', 'central government',
           'ISF/Government', 'CENTRAL GOVERNMENT', 'MINISTRY OF WATER', 
            'Ministry of water engineer', 'Cental Government', 'ADRA/Government']

list2 = ['World vision', 'World Vision', 'WORLD VISION', 'World Vission', 
        'World Division', 'Government /World Vision', 'World visiin']


df2['installer'] = df2['installer'].replace(red_list, 'government')
df2['installer'] = df2['installer'].replace(list2, 'World Vision')

testdf['installer'] = testdf['installer'].replace(red_list, 'government')
testdf['installer'] = testdf['installer'].replace(list2, 'World Vision')

In [26]:
df2.groupby(['water_quality', 'basin']).count()

id  amount_tsh  date_recorded  funder  gps_height  installer  num_private  subvillage  region_code  district_code   lga  ward  population  scheme_management  permit  construction_year  extraction_type  extraction_type_group  extraction_type_class  management  management_group  payment  quality_group  quantity  quantity_group  source  source_type  source_class  waterpoint_type  waterpoint_type_group  status_group  year_made   pop
water_quality      basin                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
coloured           Internal                   66          66             66      66          66         66           66          66           66             66    66    66          66                 66      66                 66               66                     66                     66          66                66       66             66        66              66      66           66            66               66                     66            66         66    66
                   Lake Nyasa                  7           7              7       7           7          7            7           7            7              7     7     7           7                  7       7                  7                7                      7                      7           7                 7        7              7         7               7       7            7             7                7                      7             7          7     7
                   Lake Rukwa                  8           8              8       8           8          8            8           8            8              8     8     8           8                  8       8                  8                8                      8                      8           8                 8        8              8         8               8       8            8             8                8                      8             8          8     8
                   Lake Tanganyika            64          64             64      64          64         64           64          64           64             64    64    64          64                 64      64                 64               64                     64                     64          64                64       64             64        64              64      64           64            64               64                     64            64         64    64
                   Lake Victoria              49          49             49      49          49         49           49          49           49             49    49    49          49                 49      49                 49               49                     49                     49          49                49       49             49        49              49      49           49            49               49                     49            49         49    49
                   Pangani                    35          35             35      35          35         35           35          35           35             35    35    35          35                 35      35                 35               35                     35                     35          35                35       35             35        35              35      35           35            35               35                     35            35         35    35
                   Rufiji                    183         183            183     183         183        183          183         183          183            183   183   183         183                183     183    

In [27]:
df2.groupby(['management', 'status_group']).count()

id  amount_tsh  date_recorded  funder  gps_height  installer  num_private  basin  subvillage  region_code  district_code    lga   ward  population  scheme_management  permit  construction_year  extraction_type  extraction_type_group  extraction_type_class  management_group  payment  water_quality  quality_group  quantity  quantity_group  source  source_type  source_class  waterpoint_type  waterpoint_type_group  year_made    pop
management       status_group                                                                                                                                                                                                                                                                                                                                                                                                                                                               
company          functional                 267         267            267     267         267        267          267    267         267          267            267    267    267         267                267     267                267              267                    267                    267               267      267            267            267       267             267     267          267           267              267                    267        267    267
                 functional needs repair     15          15             15      15          15         15           15     15          15           15             15     15     15          15                 15      15                 15               15                     15                     15                15       15             15             15        15              15      15           15            15               15                     15         15     15
                 non functional             403         403            403     403         403        403          403    403         403          403            403    403    403         403                403     403                403              403                    403                    403               403      403            403            403       403             403     403          403           403              403                    403        403    403
other            functional                 505         505            505     505         505        505          505    505         505          505            505    505    505         505                505     505                505              505                    505                    505               505      505            505            505       505             505     505          505           505              505                    505        505    505
                 functional needs repair     55          55             55      55          55         55           55     55          55           55             55     55     55          55                 55      55                 55               55                     55                     55                55       55             55             55        55              55      55           55            55               55                     55         55     55
                 non functional             284         284            284     284         284        284          284    284         284          284            284    284    284         284                284     284                284              284                    284                    284               284      284            284            284       284             284     284          284           284              284                    284        284    284
other - school   functional                  23          23             23      23          23         23           23     23          23           23             23     23     23          23                 23      23           

In [28]:
df2.groupby(['water_quality', 'status_group']).count()


id  amount_tsh  date_recorded  funder  gps_height  installer  num_private  basin  subvillage  region_code  district_code    lga   ward  population  scheme_management  permit  construction_year  extraction_type  extraction_type_group  extraction_type_class  management  management_group  payment  quality_group  quantity  quantity_group  source  source_type  source_class  waterpoint_type  waterpoint_type_group  year_made    pop
water_quality      status_group                                                                                                                                                                                                                                                                                                                                                                                                                                                            
coloured           functional                 246         246            246     246         246        246          246    246         246          246            246    246    246         246                246     246                246              246                    246                    246         246               246      246            246       246             246     246          246           246              246                    246        246    246
                   functional needs repair     54          54             54      54          54         54           54     54          54           54             54     54     54          54                 54      54                 54               54                     54                     54          54                54       54             54        54              54      54           54            54               54                     54         54     54
                   non functional             190         190            190     190         190        190          190    190         190          190            190    190    190         190                190     190                190              190                    190                    190         190               190      190            190       190             190     190          190           190              190                    190        190    190
fluoride           functional                 151         151            151     151         151        151          151    151         151          151            151    151    151         151                151     151                151              151                    151                    151         151               151      151            151       151             151     151          151           151              151                    151        151    151
                   functional needs repair     13          13             13      13          13         13           13     13          13           13             13     13     13          13                 13      13                 13               13                     13                     13          13                13       13             13        13              13      13           13            13               13                     13         13     13
                   non functional              36          36             36      36          36         36           36     36          36           36             36     36     36          36                 36      36                 36               36                     36                     36          36                36       36             36        36              36      36           36            36               36                     36         36     36
fluoride abandoned functional                   6           6              6       6           6          6            6      6           6            6              6      6      6           6                  6       6                  6

In [29]:
df2['status_group'].groupby(df2['water_quality']).count()

water_quality
coloured                490
fluoride                200
fluoride abandoned       17
milky                   804
salty                  4856
salty abandoned         339
soft                  50818
unknown                1876
Name: status_group, dtype: int64

In [30]:
df2.date_recorded.unique()

array(['2011-03-14', '2013-03-06', '2013-02-25', '2013-01-28',
       '2011-07-13', '2011-03-13', '2012-10-01', '2012-10-09',
       '2012-11-03', '2011-08-03', '2011-02-20', '2013-02-18',
       '2012-10-14', '2013-03-15', '2012-10-20', '2011-08-04',
       '2011-07-04', '2011-09-04', '2011-07-22', '2011-02-22',
       '2011-02-27', '2013-02-10', '2011-10-04', '2013-11-03',
       '2013-01-21', '2013-01-16', '2011-07-11', '2013-03-05',
       '2013-03-16', '2011-03-23', '2011-03-16', '2013-03-19',
       '2011-03-11', '2011-02-23', '2013-03-28', '2011-07-16',
       '2011-03-27', '2013-02-11', '2013-10-03', '2011-03-12',
       '2011-07-07', '2013-01-15', '2013-03-18', '2012-10-22',
       '2013-02-05', '2011-07-27', '2011-04-04', '2013-02-21',
       '2011-08-18', '2011-07-31', '2011-08-01', '2011-07-14',
       '2013-02-22', '2013-07-03', '2013-08-03', '2013-01-22',
       '2011-03-22', '2013-05-03', '2013-01-19', '2013-02-09',
       '2011-01-04', '2013-02-04', '2011-03-05', '2011-

In [0]:
df2['2011'] = df2['date_recorded'].str.contains('2011', na=False, regex=True)
df2['2012'] = df2['date_recorded'].str.contains('2012', na=False, regex=True)
df2['2013'] = df2['date_recorded'].str.contains('2013', na=False, regex=True)

testdf['2011'] = testdf['date_recorded'].str.contains('2011', na=False, regex=True)
testdf['2012'] = testdf['date_recorded'].str.contains('2012', na=False, regex=True)
testdf['2013'] = testdf['date_recorded'].str.contains('2013', na=False, regex=True)

In [32]:
df2['2011'] = df2['2011'].replace(True, 2011)
df2['2012'] = df2['2012'].replace(True, 2012)
df2['2013'] = df2['2013'].replace(True, 2013)

testdf['2011'] = testdf['2011'].replace(True, 2011)
testdf['2012'] = testdf['2012'].replace(True, 2012)
testdf['2013'] = testdf['2013'].replace(True, 2013)

larger = lambda s1, s2: s1 if s1.sum() > s2.sum() else s2

df2['yrs_intermediate'] = df2['2011'].combine(df2['2012'], larger)
df2['yrs_intermediate'] = df2['yrs_intermediate'].combine(df2['2013'], larger)
df2['yrs in operation'] = df2['yrs_intermediate'] - df2['construction_year']


testdf['yrs_intermediate'] = testdf['2011'].combine(testdf['2012'], larger)
testdf['yrs_intermediate'] = testdf['yrs_intermediate'].combine(testdf['2013'], larger)
testdf['yrs in operation'] = testdf['yrs_intermediate'] - testdf['construction_year']



df2.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,num_private,basin,subvillage,region_code,district_code,lga,ward,population,scheme_management,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group,year_made,pop,2011,2012,2013,yrs_intermediate,yrs in operation
0,69572,6000.0,2011-03-14,Roman,1390,Roman,0,Lake Nyasa,Mnyusi B,11,5,Ludewa,Mundindi,109,VWC,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional,3,4,2011.0,0.0,0.0,2011.0,12.0
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,0,Lake Victoria,Nyamara,20,2,Serengeti,Natta,280,Other,True,2010,gravity,gravity,gravity,wug,user-group,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional,4,5,0.0,0.0,2013.0,2013.0,3.0
2,34310,25.0,2013-02-25,Lottery Club,686,World Vision,0,Pangani,Majengo,21,4,Simanjiro,Ngorika,250,VWC,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional,4,4,0.0,0.0,2013.0,2013.0,4.0
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,0,Ruvuma / Southern Coast,Mahakamani,90,63,Nanyumbu,Nanyumbu,58,VWC,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional,2,3,0.0,0.0,2013.0,2013.0,27.0
4,19728,0.0,2011-07-13,Action In A,0,Artisan,0,Lake Victoria,Kyanyamisa,18,1,Karagwe,Nyakasimbi,0,unknown,True,0,gravity,gravity,gravity,other,other,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional,1,1,2011.0,0.0,0.0,2011.0,2011.0


In [33]:
df2['source_type'].unique()

array(['spring', 'rainwater harvesting', 'dam', 'borehole', 'other',
       'shallow well', 'river/lake'], dtype=object)

In [34]:
df2['subvillage'].value_counts()

Madukani                      508
Shuleni                       506
Majengo                       502
Kati                          373
unknown                       372
Mtakuja                       262
Sokoni                        232
M                             187
Muungano                      172
Mbuyuni                       164
Mlimani                       152
Songambele                    147
Miembeni                      134
Msikitini                     134
1                             132
Kibaoni                       114
Kanisani                      111
Mapinduzi                     109
I                             109
Mjimwema                      108
Mjini                         108
Mkwajuni                      104
Mwenge                        102
Azimio                         98
Mabatini                       98
Mbugani                        95
Mission                        95
Bwawani                        91
Bondeni                        90
Chang'Ombe    

#X and y, pipe it!

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [0]:
X = df2.drop(['status_group', 'funder', 'subvillage', 'population', 'construction_year', 'num_private',
             'region_code', 'extraction_type_group', 'quantity_group', 'source_class', 'source_type', 'subvillage',
             'permit', 'date_recorded', '2011', '2012', '2013', 'id', 'waterpoint_type_group', 'gps_height',
             'amount_tsh', 'management_group', 'district_code'], axis = 1)
y = df2['status_group']

Xtdf = testdf.drop(['funder', 'subvillage', 'population', 'construction_year', 'num_private',
                   'region_code', 'extraction_type_group', 'quantity_group', 'source_class', 'source_type',
                   'subvillage', 'permit', 'date_recorded', '2011', '2012', '2013', 'id', 'waterpoint_type_group',
                   'gps_height', 'amount_tsh', 'management_group', 'district_code'], axis = 1)

In [37]:
X.describe(include = 'all')

,installer,basin,lga,ward,scheme_management,extraction_type,extraction_type_class,management,payment,water_quality,quality_group,quantity,source,waterpoint_type,year_made,pop,yrs_intermediate,yrs in operation
count,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400.0,59400.0,59400.000000,59400.000000
unique,2116,9,125,2092,12,18,7,12,7,8,6,5,10,7,5.0,8.0,NaN,NaN
top,DWE,Lake Victoria,Njombe,Igosi,VWC,gravity,gravity,vwc,never pay,soft,good,enough,spring,communal standpipe,1.0,1.0,NaN,NaN
freq,17402,10248,2503,307,36793,26780,26780,40507,25348,50818,50818,33186,17021,28522,26464.0,28690.0,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010.875842,710.223367
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.960101,952.163833
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-2011.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011.000000,8.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012.000000,26.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013.000000,2011.000000


In [38]:
Xtdf.describe(include = 'all')

,installer,basin,lga,ward,scheme_management,extraction_type,extraction_type_class,management,payment,water_quality,quality_group,quantity,source,waterpoint_type,year_made,pop,yrs_intermediate,yrs in operation
count,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358.0,14358.0,14358.000000,14358.000000
unique,1053,9,124,1934,12,17,7,12,7,8,6,5,10,7,5.0,8.0,NaN,NaN
top,DWE,Lake Victoria,Njombe,Igosi,VWC,gravity,gravity,vwc,never pay,soft,good,enough,shallow well,communal standpipe,1.0,1.0,NaN,NaN
freq,4162,2535,611,79,8807,6168,6168,9780,6098,12237,12237,7997,4211,6790,6353.0,6972.0,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010.230673,711.978688
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.149062,954.048634
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-2011.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011.000000,8.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012.000000,26.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013.000000,2011.000000


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [40]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(testdf.shape)
print(Xtdf.shape)

(44550, 18)
(14850, 18)
(44550,)
(14850,)
(14358, 39)
(14358, 18)


In [41]:
X_train.head(1)

,installer,basin,lga,ward,scheme_management,extraction_type,extraction_type_class,management,payment,water_quality,quality_group,quantity,source,waterpoint_type,year_made,pop,yrs_intermediate,yrs in operation
2266,CES,Pangani,Hai,Machame Kaskazini,Water Board,gravity,gravity,water board,pay per bucket,soft,good,enough,spring,communal standpipe,4,1,2013.0,5.0


In [42]:
Xtdf.head(1)

,installer,basin,lga,ward,scheme_management,extraction_type,extraction_type_class,management,payment,water_quality,quality_group,quantity,source,waterpoint_type,year_made,pop,yrs_intermediate,yrs in operation
0,DMDD,Internal,Mbulu,Bashay,Parastatal,other,other,parastatal,never pay,soft,good,seasonal,rainwater harvesting,other,5,5,2013.0,1.0


In [43]:
print(X_train.dtypes)
print(y_train.dtypes)

installer                  object
basin                      object
lga                        object
ward                       object
scheme_management          object
extraction_type            object
extraction_type_class      object
management                 object
payment                    object
water_quality              object
quality_group              object
quantity                   object
source                     object
waterpoint_type            object
year_made                category
pop                      category
yrs_intermediate          float64
yrs in operation          float64
dtype: object
object


In [44]:
Xtdf.shape

(14358, 18)

In [0]:
pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True),
    RobustScaler(),
    LogisticRegression()
)

In [46]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
accuracy_score(y_test, y_pred)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.7752861952861952

In [47]:
subby = pipeline.predict(Xtdf)
print(subby.shape)
subby = pd.DataFrame(subby)
subs = id.join(subby, rsuffix = '0')
subs = subs.rename(index=str, columns={0:'status_group'})
print(subs.shape)
subs.head()

(14358,)
(14358, 2)


,id,status_group
0,50785,non functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional


In [0]:
subs.to_csv('sub1.csv', index=False)

In [0]:
y_train_e = y_train.replace({'functional':2, 'functional needs repair':1, 'non functional':0})
y_test_e = y_test.replace({'functional':2, 'functional needs repair':1, 'non functional':0})

In [0]:
pipeline3 = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True),
    RobustScaler(),
    xgb.XGBClassifier()
)



In [0]:
pipeline3.fit(X_train, y_train_e)
y_pred2 = pipeline3.predict(X_test)
print(accuracy_score(y_test_e, y_pred2))



In [0]:
xgb_pred = pipeline3.predict(Xtdf)
print(xgb_pred.shape)
xgb_pred = pd.DataFrame(xgb_pred)
subs2 = id.join(xgb_pred, rsuffix = '0')
subs2 = subs2.rename(index=str, columns={0:'status_group'})
print(subs2.shape)
subs2.head()

In [0]:
subs2.to_csv('sub2.csv', index=False)

In [0]:
param_grid = {
    'logisticregression__C': [.01, 1, 10, 1000],
    'logisticregression__class_weight':['balanced', None],
    'selectkbest__k': range(1, 6)}


gs = GridSearchCV(pipeline, param_grid=param_grid, cv=3, 
                  scoring='recall', 
                  verbose=10)
gs.fit(X_train, y_train)

In [0]:
validation_score = gs.best_score_

print('Cross-Validation Score:', validation_score)

print('Best estimator:', gs.best_estimator_)

In [0]:
selector = gs.best_estimator_.named_steps['polynomialfeatures']
all_names = X_train.columns
selected_mask = selector.get_feature_names()


print('Features selected:')
for name in selected_mask:
    print(name)

In [0]:
y_pred = gs.predict(X_test)
y_pred

In [0]:
test_score = mean_absolute_error(y_test, y_pred)
print('Test Score:', test_score)